In [1]:

!pip install nvcc4jupyter

In [2]:





%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp1hf8pu65".


Input is instantiated by all 1s

kernel is instantiated by all 2s

In [5]:

%%cuda
//parallel code implementation of cuConv for n>=1 (simple without tiling)
#include<bits/stdc++.h>
#include <sys/time.h>

#define MAX_DEPTH 1024
#define THREADS 32
#define debug 1
#define N 1
#define C 5
#define H 5
#define K 3


using namespace std;


__global__ void stage1kernel(int* d_input, int* d_filter, int* d_partial, int cf, int hf, int wf, int w, int h, int r_field_size){
      __shared__ int S[MAX_DEPTH]; //this is the shared memory
      int tid = threadIdx.x + threadIdx.y*(w-wf+1);
      if(tid < cf){ //maximum depth supported with this implementation is 1024
          S[tid] = d_filter[tid*(hf*wf) + blockIdx.y*(wf) + blockIdx.x];


      }
      __syncthreads();
      if(threadIdx.x<w-wf+1 && threadIdx.y < h-hf+1){
        d_partial[ (blockIdx.y*(wf) + blockIdx.x)*r_field_size + threadIdx.y*(w-wf+1) + threadIdx.x] = 0;
        for(int i=0;i<cf;i++){
            int iy = (blockIdx.y + threadIdx.y);
            int ix = (blockIdx.x + threadIdx.x);
            int iz = i;
            d_partial[ (blockIdx.y*(wf) + blockIdx.x)*r_field_size + threadIdx.y*(w-wf+1) + threadIdx.x] += d_input[ iz*w*h + iy*w + ix ]*S[i];
            //          ^^this is the no. of rows done ^^space each took ^^loc in a particular matrix
          }
      }

    }


__global__ void stage2kernel(int* d_partial, int n, int m, int depth){
    int stride = 1;
    int tid = threadIdx.x;
    while(stride < depth){

        if(tid%(stride*2)==0 && tid+stride < depth){

            d_partial[tid*(n*m) + blockIdx.y*n + blockIdx.x] += d_partial[(tid+stride)*(n*m) + blockIdx.y*n + blockIdx.x];
        }

      stride = stride*2;
        __syncthreads();

    }

 }


int main(){
   struct timeval begin, end;
   int *input;
   int n=N,c=C,h=H,w=H; //only doing for n=1 now
   int input_size = n*c*h*w;
   input=(int*)malloc(input_size*sizeof(int));

   int *filter;
   int cf=C,hf=K,wf=K;
   int filter_size = cf*hf*wf;
   filter=(int*)malloc(filter_size*sizeof(int));


   //initializing the 4D input
   for(int i=0;i<n;i++){
       for(int j=0;j<c;j++){
           for(int k=0;k<h;k++){
               for(int l=0;l<w;l++){
                   input[ (c*h*w)*i + (h*w)*j + (w)*k + l ] = 1;
               }
           }
       }
   }

   //initializing the 3D filter
   for(int i=0;i<cf;i++){
       for(int j=0;j<hf;j++){
           for(int k=0;k<wf;k++){
              filter[ (hf*wf)*i + (wf)*j + k ] = 2;
           }
       }
   }
   int *d_input, *d_filter;
    int filter_rows = hf*wf;
    int r_field_size = (w-wf+1)*(h-hf+1);

    cudaMalloc(&d_input,c*h*w*sizeof(int));
    cudaMalloc(&d_filter,filter_size*sizeof(int));

    cudaMemcpy(d_filter, filter , filter_size * sizeof(int), cudaMemcpyHostToDevice);
    int* d_partial;
    cudaMalloc(&d_partial,filter_rows*r_field_size*sizeof(int));
    dim3 grid(wf,hf);
    dim3 block(THREADS, THREADS);
    int* output;
    output=(int*)malloc(n*r_field_size*sizeof(int));

    gettimeofday(&begin, 0);
   for(int i=0;i<n;i++){
    cudaMemcpy(d_input, input +i*c*h*w, c*h*w* sizeof(int), cudaMemcpyHostToDevice);
    stage1kernel<<<grid, block>>>(d_input, d_filter, d_partial, cf, hf, wf, w, h, r_field_size);
    cudaDeviceSynchronize();

    dim3 grid2(w-wf+1,h-hf+1);
    int threads2 = filter_rows;

    stage2kernel<<<grid2,threads2 >>>(d_partial, w-wf+1, h-hf+1, filter_rows);
    cudaDeviceSynchronize();
    cudaMemcpy(output + i*(w-wf+1)*(h-hf+1), d_partial , r_field_size * sizeof(int), cudaMemcpyDeviceToHost);

    }
     gettimeofday(&end, 0);


  if(debug){
   //display the convolution results
   for(int l=0;l<n;l++){
      for(int i=0;i<h-hf+1;i++){
          for(int j=0;j<w-wf+1;j++){
              cout<<output[l*(w-wf+1)*(h-hf+1) + (w-wf+1)*i + j]<<" ";
          }
          cout<<"\n";
      }
      cout<<"####\n";
  }
}
    long seconds = end.tv_sec - begin.tv_sec;
    long microseconds = end.tv_usec - begin.tv_usec;
    double elapsed = seconds + microseconds*1e-6;
    printf("Time measured: %.6f seconds.\n", elapsed);
    return 0;
}


90 90 90 
90 90 90 
90 90 90 
####
Time measured: 0.000264 seconds.



Please check output in C.txt

In [8]:

%%cuda
//parallel code implementation of cuConv for n>=1, supporting bigger input with tiling implementation
#include<bits/stdc++.h>
#include <sys/time.h>

#define MAX_DEPTH 1024
#define THREADS 32
#define debug 1
#define N 1
#define C 25
#define H 30
#define K 1
#define TILE_WIDTH 32

using namespace std;


__global__ void stage1kernel(int* d_input, int* d_filter, int* d_partial, int cf, int hf, int wf, int w, int h, int r_field_size){
      __shared__ int S[MAX_DEPTH]; //this is the shared memory
      int tid = threadIdx.x + threadIdx.y*TILE_WIDTH;
      if(tid < cf){ //maximum depth supported with this implementation is 1024
          S[tid] = d_filter[tid*(hf*wf) + blockIdx.y*(wf) + blockIdx.x];

      }
      __syncthreads();

      for(int y=0;y<h-hf+1;y+=TILE_WIDTH){
          for(int x=0;x<w-wf+1;x+=TILE_WIDTH){
              int tx = x + threadIdx.x;
              int ty = y + threadIdx.y;
              if(tx<w-wf+1 && ty < h-hf+1){
                d_partial[ (blockIdx.y*(wf) + blockIdx.x)*r_field_size + ty*(w-wf+1) + tx] = 0;
                for(int i=0;i<cf;i++){
                    int iy = (blockIdx.y + ty);
                    int ix = (blockIdx.x + tx);
                    int iz = i;


                    d_partial[ (blockIdx.y*(wf) + blockIdx.x)*r_field_size + ty*(w-wf+1) + tx] += d_input[ iz*w*h + iy*w + ix ]*S[i];
                    //          ^^this is the no. of rows done ^^space each took ^^loc in a particular matrix


                  }
              }



          }

      }

    }


__global__ void stage2kernel(int* d_partial, int n, int m, int depth){


    for(int x=0;x*TILE_WIDTH<n;x++){
        for(int y=0;y*TILE_WIDTH<m;y++){
            int bx = x*TILE_WIDTH + blockIdx.x, by = y*TILE_WIDTH + blockIdx.y;
            if(bx<n && by < m){

                int stride = 1;
                int tid = threadIdx.x;
                while(stride < depth){


                    if(tid%(stride*2)==0 && tid+stride < depth){

                        d_partial[tid*(n*m) + by*n + bx] += d_partial[(tid+stride)*(n*m) + by*n + bx];
                    }
                  stride = stride*2;
                    __syncthreads();



                }
            }

        }
    }

 }


int main(){
    freopen("C.txt", "w", stdout); //for writing to C.txt using cout

   struct timeval begin, end;
   int *input;
   int n=N,c=C,h=H,w=H; //only doing for n=1 now
   int input_size = n*c*h*w;
   input=(int*)malloc(input_size*sizeof(int));

   int *filter;
   int cf=C,hf=K,wf=K;
   int filter_size = cf*hf*wf;
   filter=(int*)malloc(filter_size*sizeof(int));


   //initializing the 4D input
   for(int i=0;i<n;i++){
       for(int j=0;j<c;j++){
           for(int k=0;k<h;k++){
               for(int l=0;l<w;l++){
                   input[ (c*h*w)*i + (h*w)*j + (w)*k + l ] = 1;
               }
           }
       }
   }

   //initializing the 3D filter
   for(int i=0;i<cf;i++){
       for(int j=0;j<hf;j++){
           for(int k=0;k<wf;k++){
              filter[ (hf*wf)*i + (wf)*j + k ] = 2;
           }
       }
   }
   int *d_input, *d_filter;
    int filter_rows = hf*wf;
    int r_field_size = (w-wf+1)*(h-hf+1);

    cudaMalloc(&d_input,c*h*w*sizeof(int));
    cudaMalloc(&d_filter,filter_size*sizeof(int));

    cudaMemcpy(d_filter, filter , filter_size * sizeof(int), cudaMemcpyHostToDevice);
    dim3 grid(wf,hf);
    dim3 block(TILE_WIDTH, TILE_WIDTH);
    int* output;
    output=(int*)malloc(n*r_field_size*sizeof(int));

    gettimeofday(&begin, 0);
   for(int i=0;i<n;i++){
    int* d_partial;
    cudaMalloc(&d_partial,filter_rows*r_field_size*sizeof(int));
    cudaMemcpy(d_input, input +i*c*h*w, c*h*w* sizeof(int), cudaMemcpyHostToDevice);

    stage1kernel<<<grid, block>>>(d_input, d_filter, d_partial, cf, hf, wf, w, h, r_field_size);
    cudaDeviceSynchronize();

    dim3 grid2(TILE_WIDTH,TILE_WIDTH);
    int threads2 = filter_rows;
    stage2kernel<<<grid2,threads2 >>>(d_partial, w-wf+1, h-hf+1, filter_rows);
    cudaDeviceSynchronize();
    cudaMemcpy(output + i*(w-wf+1)*(h-hf+1), d_partial , r_field_size * sizeof(int), cudaMemcpyDeviceToHost);

    }
     gettimeofday(&end, 0);


  // check for error
  cudaError_t error = cudaGetLastError();
  if(error != cudaSuccess)
  {
    // print the CUDA error message and exit
    printf("CUDA error: %s\n", cudaGetErrorString(error));
    exit(-1);
  }



  if(debug){
   //display the convolution results
   for(int l=0;l<n;l++){
      for(int i=0;i<h-hf+1;i++){
          for(int j=0;j<w-wf+1;j++){
              cout<<output[l*(w-wf+1)*(h-hf+1) + (w-wf+1)*i + j]<<" ";
          }
          cout<<"\n";
      }
      cout<<"####\n";
  }
}
    long seconds = end.tv_sec - begin.tv_sec;
    long microseconds = end.tv_usec - begin.tv_usec;
    double elapsed = seconds + microseconds*1e-6;
    printf("Time measured: %.6f seconds.\n", elapsed);
    return 0;
}
